In [6]:
!pip install openpyxl

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
# params 

# Path : Top Project Path 
path = r'C:\Users\Tmax\Desktop\Hyperdata_web_TOP3'  
path = path.replace('\\','\\' )

## TOP 프로젝트에서 Object 별 String 추출
- JS 파일에서 string 외에 script 가 같이 추출되는 현상 해결 필요
- 중복 제거해서 output? output 후 중복 제거?
- {} 안에 들어가는 바인딩 요소
- @string으로 시작하는 String ID 부여한 내용은 제외함

In [1]:
import os

# Get File list function 
# 특정 포멧의 파일 리스트 [위치, 폴더, 파일명] 생성
def get_file_list(path, file_format):
    file_list =[]
    if file_format == '.js':
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(file_format) and root.startswith(path+'\\src' ):
                    folder = root.split('\\')[-2:]
                    file_list.append([root,folder[0]+'\\'+folder[1],file])
                else: pass
    else:
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(file_format):
                    folder = root.split('\\')[-2:]
                    file_list.append([root,folder[0]+'\\'+folder[1],file])
                else: pass
    return file_list

In [2]:
def is_text(input_s):
    if input_s.startswith("@string"): #String 지정된 값 pass
        return False
    elif input_s.startswith("{") and input_s.endswith("}"):
        return False
    
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣'):
            return True
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            return True
    return False

In [3]:
#get id, properties from files 
def get_id_properties(file_type,file_list):
    prop_list=[]
    i = 1 #index
    
    for file in file_list:
        file_path = file[0] + '\\'
        file_name = file[2]
        
        code = open(file_path+file_name,'rt', encoding='UTF8')
        while True:
            line = code.readline()
            if not line: break
            line = line.strip()
            if line.startswith("id="):
                object_id=line
            if line.startswith("text=") or line.startswith("hint=") or line.startswith("title=") or line.startswith("message="):
                object_id=object_id.replace("\"","").replace('id=','')
                text_type=line.split('=')[0]
                string = line.split('=')[1].replace("\"","").replace(">","")
                
                if is_text(string):
                    prop_list.append([i,file_type,"",file[1],file_name,object_id,text_type,string])
                    i+=1
                
                
        code.close()
        
    return prop_list

In [4]:
# Write List to Excel file
import openpyxl

def write_xl(filename, string_list):
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.cell(row=1,column=1).value="Index"
    sheet.cell(row=1,column=2).value="Type"
    sheet.cell(row=1,column=3).value="StringID"
    sheet.cell(row=1,column=4).value="Path"
    sheet.cell(row=1,column=5).value="File"
    sheet.cell(row=1,column=6).value="ObjectID // Function"
    sheet.cell(row=1,column=7).value="TextType"
    sheet.cell(row=1,column=8).value="TextValue"
    
    sheet.column_dimensions['B'].width= 30
    sheet.column_dimensions['C'].width= 30
    sheet.column_dimensions['D'].width= 40
    sheet.column_dimensions['E'].width= 50
    sheet.column_dimensions['f'].width= 50
    sheet.column_dimensions['g'].width= 20
    sheet.column_dimensions['h'].width= 70
    
    for i in range(len(string_list)):
        sheet.cell(i+2,1).value=string_list[i][0]
        sheet.cell(i+2,2).value=string_list[i][1]
        sheet.cell(i+2,3).value=string_list[i][2]
        sheet.cell(i+2,4).value=string_list[i][3]
        sheet.cell(i+2,5).value=string_list[i][4]
        sheet.cell(i+2,6).value=string_list[i][5]
        sheet.cell(i+2,7).value=string_list[i][6]
        sheet.cell(i+2,8).value=string_list[i][7]
    
    wb.save(filename+".xlsx")
    wb.close()    
    
    

In [5]:
## JS Logic settext parshing
def get_js_func_properties(file_list):
    prop_list=[]
    i=1

    for file in js_list:
        file_path = file[0] + '\\'
        file_name = file[2]

        code = open(file_path+file_name,'rt', encoding='UTF8')
        while True:
            line = code.readline()
            if not line: break
            line = line.strip()

#             if line.startswith("function"):
#                 func = line

            if 'Top.Dom.selectById("' in line:
                object_id = line.split('Top.Dom.selectById("')[1]

            if '.setText' in line:
                string = line.split('.setText')[-1]
                object_id=object_id.replace("\"","") 
#                 string = string.replace("(","").replace(";","").replace(")","")
                if '\"' in string :
                    string_ = string.split("\"")
                    for j in range(len(string_)):
                        if j%2 == 1 and is_text(string_[j]):
                            prop_list.append([i,'js','',file[1],file_name,object_id,'setText',string_[j]])
                            i+=1

        code.close()

    return prop_list

In [39]:
#1. Runtime Files
path2 = path +'\\meta\\runtime'
runtime_files = get_file_list(path2, '.xml')
runtime_text = get_id_properties('runtime',runtime_files)

In [40]:
#2 CustomWidget FILES
path3 = path +'\\meta'
custom_files = get_file_list(path3, 'tcw')
custom_text = get_id_properties('custom',custom_files)
write_xl('custom',custom_text)

In [41]:
# 3.menu files
menu_files= get_file_list(path, 'tmrf')
menu_text = get_id_properties('menu',menu_files)
write_xl('menu',menu_text)

In [42]:
# 4.layout files
layout_files= get_file_list(path, 'tlf')
layout_text = get_id_properties('layout',layout_files)
write_xl('layout',layout_text)

In [101]:
# 5. js files
js_list=get_file_list(path, '.js')
js_text = get_js_func_properties(js_list)
write_xl('js3',js_text)

In [100]:
js_list

[['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\src\\com\\tmax\\hyperdata\\js\\aimodel',
  'js\\aimodel',
  'AIModelCodeDialLayoutLogic.js'],
 ['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\src\\com\\tmax\\hyperdata\\js\\aimodel',
  'js\\aimodel',
  'AIModelDialLayoutLogic.js'],
 ['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\src\\com\\tmax\\hyperdata\\js\\aimodel',
  'js\\aimodel',
  'AIModelLogic.js'],
 ['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\src\\com\\tmax\\hyperdata\\js\\datasource',
  'js\\datasource',
  'DataAjaxRequest.js'],
 ['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\src\\com\\tmax\\hyperdata\\js\\datasource',
  'js\\datasource',
  'DataCommonLogic.js'],
 ['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\src\\com\\tmax\\hyperdata\\js\\datasource',
  'js\\datasource',
  'DataObjectDetailLogic.js'],
 ['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\src\\com\\tmax\\hyperdata\\js\\datasource',
  'js\\datasource',
  'DataObjectInfoLogic.js'],
 ['C:\\Users\\Tmax\\Desktop

In [6]:
def top_str_extract_main(path,excel_file_name):
    path = path.replace('\\','\\')
    
    #1. Runtime Files
    path2 = path +'\\meta\\runtime'
    runtime_files = get_file_list(path2, '.xml')
    runtime_text = get_id_properties('runtime',runtime_files)
    
    #2 CustomWidget FILES
    path3 = path +'\\meta'
    custom_files = get_file_list(path3, 'tcw')
    custom_text = get_id_properties('custom',custom_files)
    
    # 3.menu files
    menu_files= get_file_list(path, 'tmrf')
    menu_text = get_id_properties('menu',menu_files)
    
    # 4.layout files
    layout_files= get_file_list(path, 'tlf')
    layout_text = get_id_properties('layout',layout_files)
    
    # 5. js files
    js_list = get_file_list(path, '.js')
    if js_list != [] :
        js_text = get_js_func_properties(js_list)
    else:
        js_text = []
    
    all_list = menu_text+custom_text+runtime_text+layout_text+js_text
    
    write_xl(excel_file_name, all_list)

In [19]:
js_list=get_file_list(r'C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest'  , '.js')

In [20]:
js_list

[['C:\\Users\\Tmax\\Desktop\\cloudworkspace\\TopStringTest\\src\\common',
  'src\\common',
  'script.js']]

In [97]:
top_str_extract_main(r'C:\Users\Tmax\Desktop\Hyperdata_web_TOP3','hyperdata_string')

NameError: name 'js_list' is not defined

In [98]:
get_file_list(r'C:\\Users\\Tmax\\Desktop\\cloudworkspace\\TopStringTest', '.js')

[['C:\\\\Users\\\\Tmax\\\\Desktop\\\\cloudworkspace\\\\TopStringTest\\src\\common',
  'src\\common',
  'script.js']]

In [92]:
top_str_extract_main(r'C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest','test_string')

NameError: name 'js_list' is not defined

## String 파일 추출 - id/string_value

In [7]:
# 프로젝트 하위 String 파일 목록 추출
def get_string_file_list(path):
    file_list =[]
    path = path +'\\res'
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.startswith('str'):
                file_list.append(root+'\\'+file)
    return file_list

In [34]:
get_string_file_list(r'C:\Users\Tmax\Desktop\Hyperdata_web_TOP3')

['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\res\\values\\strings\\str.xml',
 'C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\res\\values-ko\\strings\\str.xml']

In [8]:
def read_string_file(file_path):
    string_list = []
    code = open(file_path, 'rt', encoding='UTF8')
    while True:
        line = code.readline()
        if not line: break
        line = line.strip()
        if line.startswith("name="):
            str_id = line.split("\"")[1]
            str_val = line.split('>')[1].split('<')[0]
            string_list.append([str_id,str_val])
        else: pass
        
    return string_list
    

    

In [36]:
read_string_file('C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\res\\values\\strings\\str.xml')

[['str_name', 'Name'],
 ['str_team', 'Team'],
 ['str_user', 'User'],
 ['str_age', 'Age']]

In [9]:
#영문, 한글 두 str파일을 파싱하여 하나의 dict로 생성 
# key: id , value: string value list
from collections import defaultdict

def extract_match_string(files):
    # 파일별 string 추출
    file_dict = defaultdict(list)
    for file in files:
        file_dict[file] = read_string_file(file)
   

    string_dict = defaultdict(list)
    for file in file_dict :
        for string in file_dict[file]:
            str_id = string[0]
            str_val = string[1]
            if string_dict[str_id]:
                string_dict[str_id].append(str_val)
            else :
                string_dict[str_id]=[str_val]
    
    return string_dict
    

In [19]:
str_dict = extract_match_string(['C:\\Users\\Tmax\\Desktop\\cloudworkspace\\TopStringTest\\res\\values\\strings\\string_.xml',
'C:\\Users\\Tmax\\Desktop\\cloudworkspace\\TopStringTest\\res\\values-ko\\strings\\string_.xml'])

In [56]:
str_dict = extract_match_string(['C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\res\\values\\strings\\str.xml',
 'C:\\Users\\Tmax\\Desktop\\Hyperdata_web_TOP3\\res\\values-ko\\strings\\str.xml'])

In [20]:
str_dict

defaultdict(list,
            {'String0': ['human', '사람'],
             'String1': ['apple', '사과'],
             'String2': ['tree', '나무']})

In [57]:
for value in str_dict:
    print(str_dict[value])

['Name', '이름']
['Team', '팀']
['User', '사용자']
['Age', '나이']


In [58]:
str_dict

defaultdict(list,
            {'str_name': ['Name', '이름'],
             'str_team': ['Team', '팀'],
             'str_user': ['User', '사용자'],
             'str_age': ['Age', '나이']})

In [10]:
# Write List to Excel file
import openpyxl

def write_string_xl(filename, string_dict):
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.cell(row=1,column=1).value="Index"
    sheet.cell(row=1,column=2).value="StringID"
    sheet.cell(row=1,column=3).value="en"
    sheet.cell(row=1,column=4).value="kr"
    
    sheet.column_dimensions['B'].width= 30
    sheet.column_dimensions['C'].width= 30
    sheet.column_dimensions['D'].width= 40
    
    n=1
    for str_id in string_dict:
        sheet.cell(n+1,1).value=n
        sheet.cell(n+1,2).value=str_id
        sheet.cell(n+1,3).value=string_dict[str_id][0]
        sheet.cell(n+1,4).value=string_dict[str_id][1]
        n+=1
    
    wb.save(filename+".xlsx")
    wb.close()  

In [68]:
write_string_xl('string',str_dict)

In [119]:
excel_file_name = 'hyperdata_str_raw'

In [121]:
raw_string_path = './'+excel_file_name+'.xlsx'
print(raw_string_path)

./hyperdata_str_raw.xlsx


## raw string 엑셀 읽어와서 string object 업데이트

In [147]:
string_file_list= get_file_list(string_object_path, 'string_.xml')

In [148]:
files = []
for file in string_file_list:
    files.append(file[0]+'//'+file[2])

In [ ]:
str_dict = extract_match_string(files)

In [11]:
def update_string_object(raw_string_path,string_object_path):
    
    string_file_list= get_file_list(string_object_path, 'string_.xml')
    files = []
    for file in string_file_list:
        files.append(file[0]+'//'+file[2])
        
    str_dict = extract_match_string(files)
    
    raw_string_wb = openpyxl.load_workbook(raw_string_path, data_only=True)
    raw_string_ws = raw_string_wb['Sheet']
    
    #raw_string save as list
    raw_string_data =[]
    for row in raw_string_ws:
        row_data =[]
        for cell in row:
            row_data.append(cell.value)
        raw_string_data.append(row_data)
    
    #raw string 추가
    for i in range(1,len(raw_string_data)):
        raw_string_value = raw_string_data[i][7]
        exist = isExist(raw_string_value,str_dict)
        if exist==False :
            # str_dict에 value 추가 & ID 부여
            index=len(str_dict)
            newid='String'+str(index) #id rule... need update....
            str_dict[newid]=[raw_string_data[i][7],raw_string_data[i][8]]

    for i in range(2):
        code = open(string_file_list[i][0]+'\\'+string_file_list[i][2],'r+', encoding='UTF8')
        lines=code.readlines()
        code.close()
        lines = lines[:4]
        for item in str_dict:
            lines.append('        <resource:String\n')
            stridval='        name="'+item+'">'+str_dict[item][i]+'</resource:String>\n'
            lines.append(stridval)
        lines.append('</resource:Strings>')

        code = open(string_file_list[i][0]+'\\'+string_file_list[i][2],'w+t', encoding='UTF8')
        code.writelines(lines)
        code.close()

In [13]:
raw_string_path= r"D:\\PYTHON\\TOP_string_parshing\\test_string.xlsx"
string_object_path= r'C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest\res'

In [16]:
update_string_object(raw_string_path,string_object_path)

In [27]:
raw_string_path = "D:\\PYTHON\\TOP_string_parshing\\test_string.xlsx"
# raw_string 엑셀 읽어오기
raw_string_wb = openpyxl.load_workbook(raw_string_path, data_only=True)
raw_string_ws = raw_string_wb['Sheet']

In [37]:
raw_string_data =[]
for row in raw_string_ws:
    row_data =[]
    for cell in row:
        row_data.append(cell.value)
    raw_string_data.append(row_data)
    
        

In [12]:
#string dict에 string 존재 여부 확인
def isExist(string,string_dict):
    for item in string_dict:
        if string in string_dict[item]:
            return True
    return False

In [56]:
index=0
for i in range(1,len(raw_string_data)):
    raw_string_value = raw_string_data[i][7]
    exist = isExist(raw_string_value,str_dict)
    if exist==False :
        # str_dict에 value 추가 & ID 부여
        index=len(str_dict)
        newid='String'+str(index) #id rule... need update....
        str_dict[newid]=[raw_string_data[i][7],raw_string_data[i][8]]

In [57]:
str_dict

defaultdict(list,
            {'String0': ['human', '사람'],
             'String1': ['apple', '사과'],
             'String2': ['tree', '나무'],
             'String3': ['ItemA', '아이템A'],
             'String4': ['ItemB', '아이템B'],
             'String5': ['ItemC', '아이템C'],
             'String6': ['ItemD', '아이템D'],
             'String7': ['ItemE', '아이템E'],
             'String8': ['this is notification 1', '이것은 노티피케이션 1 입니다'],
             'String9': ['notification 1', '노티피케이션 1'],
             'String10': ['this is notification 2', '이것은 노티피케이션 2 입니다'],
             'String11': ['notification 2', '노티피케이션 2'],
             'String12': ['Open Notification 1', '노티피케이션 1 열기'],
             'String13': ['Open Notification 2', '노티피케이션 2 열기'],
             'String14': ['Open Dialog', '다이얼로그 열기'],
             'String15': ['Open Popover', '팝오버 열기'],
             'String16': ['User', '사용자'],
             'String17': ['Admin', '관리자'],
             'String18': ['this is Dialog', '이것은 다이얼로그입니다'],
    

In [40]:
# Str_dict 를 string Object로 업데이트

# 기존 object 열기
string_file_list= get_file_list('C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest\res', 'string_.xml')
code = open(string_file_list[0][0]+string_file_list[0][2],'rt', encoding='UTF8')
while True:
    line = code.readline()
    if not line: break
    if line.startswith("</resource:Strings>"):
        line

String0
String1
String2


In [61]:
string_file_list= get_file_list(r'C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest\res', 'string_.xml')

In [62]:
string_file_list

[['C:\\Users\\Tmax\\Desktop\\cloudworkspace\\TopStringTest\\res\\values\\strings',
  'values\\strings',
  'string_.xml'],
 ['C:\\Users\\Tmax\\Desktop\\cloudworkspace\\TopStringTest\\res\\values-ko\\strings',
  'values-ko\\strings',
  'string_.xml']]

In [88]:
code = open(string_file_list[0][0]+'\\'+string_file_list[0][2],'r+', encoding='UTF8')
lines=code.readlines()
code.close()

lines = lines[:4]
for item in str_dict:
    lines.append('        <resource:String\n')
    stridval='        name="'+item+'">'+str_dict[item][0]+'</resource:String>\n'
    lines.append(stridval)
lines.append('</resource:Strings>')

In [89]:
lines

['<resource:Strings\n',
 '    xmlns:resource="http://www.tmaxsoft.com/top/SNAPSHOT/resource"\n',
 '    id="string_"\n',
 '    ideUniqueId="IDba060d8b430835e4b282fd2b8050765b">\n',
 '        <resource:String\n',
 '        name="String0">human</resource:String>\n',
 '        <resource:String\n',
 '        name="String1">apple</resource:String>\n',
 '        <resource:String\n',
 '        name="String2">tree</resource:String>\n',
 '        <resource:String\n',
 '        name="String3">ItemA</resource:String>\n',
 '        <resource:String\n',
 '        name="String4">ItemB</resource:String>\n',
 '        <resource:String\n',
 '        name="String5">ItemC</resource:String>\n',
 '        <resource:String\n',
 '        name="String6">ItemD</resource:String>\n',
 '        <resource:String\n',
 '        name="String7">ItemE</resource:String>\n',
 '        <resource:String\n',
 '        name="String8">this is notification 1</resource:String>\n',
 '        <resource:String\n',
 '        name="S

In [90]:
code = open(string_file_list[0][0]+'\\'+string_file_list[0][2],'w+t', encoding='UTF8')
code.writelines(lines)
code.close()

## ID 생성 및 부여.. (구체화 필요)

In [62]:
text_id_dict= dict()
text_set = set()

In [63]:
# 전체 String을 set에 저장
for i in range(len(all_list)):
    text_set.add(all_list[i][6])

In [64]:
text_set = list(text_set)
id_list = []

In [65]:
# id 부여 
for i in range(len(text_set)):
    text_id_dict[text_set[i]] = 'str_'+"%06d"% i
    id_list.append('str_'+"%06d"% i)

In [66]:
text_id_dict

{'': 'str_000000',
 '{nulls}': 'str_000001',
 'Delete Virtual Model &amp; Virtual Table': 'str_000002',
 'AND NOT': 'str_000003',
 'Threshold Line': 'str_000004',
 '{warning}': 'str_000005',
 'Left Column': 'str_000006',
 '{description}': 'str_000007',
 '{sourceTableName}': 'str_000008',
 'Cross Validation': 'str_000009',
 '!': 'str_000010',
 'Number of Param Servers': 'str_000011',
 'Every Day': 'str_000012',
 'curFiles + "\\n====Added Files====\\n" + fileNames': 'str_000013',
 'Edit User': 'str_000014',
 'Content(Json Format)': 'str_000015',
 'E-MAIL': 'str_000016',
 'Detail': 'str_000017',
 '{row}': 'str_000018',
 'Size': 'str_000019',
 'NEXT TIME': 'str_000020',
 '+ All Columns': 'str_000021',
 'Top.Dom.selectById"NewVirtualModelDescriptionTextArea".getText': 'str_000022',
 'Dashboard name': 'str_000023',
 'Add User': 'str_000024',
 'times': 'str_000025',
 'Project': 'str_000026',
 'NOT IN': 'str_000027',
 '{primaryKey}': 'str_000028',
 'Updated': 'str_000029',
 '#modelname': 'str_

In [ ]:
for i in id_list :
    text = text_id_dict[i] 

In [67]:
for item in all_list :
    text = item[6]
    item.insert(2,text_id_dict[text])
    

In [68]:
all_list

[[1,
  'menu',
  'str_000577',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'bar',
  'text',
  '  Bar Chart'],
 [2,
  'menu',
  'str_000219',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'bar1',
  'text',
  'Measure Comparison'],
 [3,
  'menu',
  'str_000869',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'bar2',
  'text',
  'Dimension Comparison'],
 [4,
  'menu',
  'str_000123',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'line',
  'text',
  '  Line Chart'],
 [5,
  'menu',
  'str_000219',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'line1',
  'text',
  'Measure Comparison'],
 [6,
  'menu',
  'str_000869',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'line2',
  'text',
  'Dimension Comparison'],
 [7,
  'menu',
  'str_000223',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'pie',
  'text',
  '  Pie Chart'],
 [8,
  'menu',
  'str_000219',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'pie1',
  'text',
  'Measure Comparison'],
 [9,
  'menu',
  'str_000869',
  'meta\\menuitem',
  'ChartMenu.tmrf',
  'pie2',
  'text

In [70]:

write_xl('all_id', all_list)

In [22]:
print ("%05d"% 5)
print (format(5000,'05')) #2.6이상
print ('{0:05d}'.format(273)) 

00005
05000
00273


In [24]:
"%06d"% 22

'00022'

In [63]:
def update_string_object(raw_string_path,path,string_object_name):
    
    string_object_path=path+'\\res'
    string_file_list= get_file_list(string_object_path, string_object_name+'.xml')
    files = []
    for file in string_file_list:
        files.append(file[0]+'//'+file[2])
        
    str_dict = extract_match_string(files)
    
    raw_string_wb = openpyxl.load_workbook(raw_string_path, data_only=True)
    raw_string_ws = raw_string_wb['Sheet']
    
    print("file read")
    
    #raw_string save as list
    raw_string_data =[]
    for row in raw_string_ws:
        row_data =[]
        for cell in row:
            row_data.append(cell.value)
        raw_string_data.append(row_data)
        
    print("create string list")
    print("------------------------------------------")
    
    #raw string 추가
    for j in range(1,len(raw_string_data)):
        raw_string_value = raw_string_data[j][7]
        exist = isExist(raw_string_value,str_dict)
        if exist==False :
            # str_dict에 value 추가 & ID 부여
            index=len(str_dict)
            newid='String'+("%06d"% index) #id rule... need update....
            str_dict[newid]=[raw_string_data[j][7],raw_string_data[j][8]]
            print(newid,raw_string_data[j][7],raw_string_data[j][8])
            
    
    print("------------------------------------------")
    
    for i in range(2):
        code = open(string_file_list[i][0]+'\\'+string_file_list[i][2],'r+', encoding='UTF8')
        lines=code.readlines()
        code.close()
        lines = lines[:4]
        for item in str_dict:
            lines.append('        <resource:String\n')
            stridval='        name="'+item+'">'+str_dict[item][i]+'</resource:String>\n'
            lines.append(stridval)
        lines.append('</resource:Strings>')
        code = open(string_file_list[i][0]+'\\'+string_file_list[i][2],'w+t', encoding='UTF8')
        code.writelines(lines)
        code.close()
        print(string_file_list[i][0]+'\\'+string_file_list[i][2])

In [64]:
path = r'C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest'  
raw_string_path=r'.\test_str_raw.xlsx'

In [65]:
update_string_object(raw_string_path,path,'string_')

file read
create string list
------------------------------------------
------------------------------------------
C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest\res\values\strings\string_.xml
C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest\res\values-ko\strings\string_.xml


In [ ]:
#String object 읽어오기, default만...

#모든 obj 파일 리스트

#개별 object 파싱 

#string value --> ID 교체 
#다시 object 쓰기

In [67]:
#String object 읽어오기, default만...
def get_string_list(path,string_object_name):
    string_object_path=path+'\\res'
    string_file_list= get_file_list(string_object_path, string_object_name+'.xml')
    files = []
    for file in string_file_list:
        files.append(file[0]+'//'+file[2])
        
    str_dict = extract_match_string(files)
    print("------------------------------------------")
    
    return str_dict

In [70]:
str_dict=get_string_list(path,'string_')

------------------------------------------


In [69]:
"/"

'/'

In [115]:
#모든 obj의 string value 파악 extract_main 변형
#개별 파일 단위로 변환
def change_string_val_to_id(file,str_dict,string_object):
    code = open(file,'r+',encoding='UTF8')
    lines = code.readlines()
    code.close()
    
    newlines=[]
    for line in lines:
        if 'text=\"' in line:
            text_val=line.split('\"')[1]
            for key, value in str_dict.items():
                if text_val == value[0] and not text_val.startswith("@string/"):
                    print(line)
                    line = line.replace(text_val,"@string/"+string_object+"/"+key)
                    print(line)
        newlines.append(line)
    code = open(file,'w+t', encoding='UTF8')
    code.writelines(newlines)
    code.close()

In [105]:
file=r'C:\Users\Tmax\Desktop\cloudworkspace\TopStringTest\meta\layout\MainTopStringTestLayout.tlf'

In [106]:
change_string_val_to_id(file,str_dict,"string_")

            text="Open Notification 1">

            text="@string/string_/String000012">

            text="Open Notification 2">

            text="@string/string_/String000013">

            text="Open Dialog">

            text="@string/string_/String000014">

            text="Open Popover">

            text="@string/string_/String000015">

            text="User">

            text="@string/string_/String000016">

            text="Admin">

            text="@string/string_/String000017">

            text="apple">

            text="@string/string_/String1">



In [116]:
def switch_all_file_string_to_id(path,string_object_name):
    
    #String object 읽어오기
    string_object_path=path+'\\res'
    string_file_list= get_file_list(string_object_path, string_object_name+'.xml')
    files = []
    for file in string_file_list:
        files.append(file[0]+'//'+file[2])
        
    str_dict = extract_match_string(files)
    
    #모든 obj 파일 리스트
    #js 파일 제외
    
    #1. Runtime Files
    path2 = path +'\\meta\\runtime'
    runtime_files = get_file_list(path2, '.xml')
    #2 CustomWidget FILES
    path3 = path +'\\meta'
    custom_files = get_file_list(path3, 'tcw')
    # 3.menu files
    menu_files= get_file_list(path, 'tmrf')
    # 4.layout files
    layout_files= get_file_list(path, 'tlf')
    
    object_files = runtime_files+custom_files+menu_files+layout_files
    
    for file in object_files:
        change_string_val_to_id(file[0]+'\\'+file[2], str_dict, string_object_name)

In [117]:
switch_all_file_string_to_id(path,'string_')

        text="ItemA">

        text="@string/string_/String000003">

        text="ItemB">

        text="@string/string_/String000004">

        text="ItemC">

        text="@string/string_/String000005">

        text="ItemD">

        text="@string/string_/String000006">

        text="ItemE">

        text="@string/string_/String000007">

            text="Open Notification 1">

            text="@string/string_/String000012">

            text="Open Notification 2">

            text="@string/string_/String000013">

            text="Open Dialog">

            text="@string/string_/String000014">

            text="Open Popover">

            text="@string/string_/String000015">

            text="User">

            text="@string/string_/String000016">

            text="Admin">

            text="@string/string_/String000017">

            text="apple">

            text="@string/string_/String1">

        text="this is Dialog"

        text="@string/string_/String000018"

     

In [ ]:
def change_string_val_to_id(file,str_dict,string_object):
    code = open(file,'r+',encoding='UTF8')
    lines = code.readlines()
    code.close()
    
    newlines=[]
    for line in lines:
        if 'text=\"' in line or 'hint=\"' in line or 'message=\"' in line or 'title=\"' in line :
            type_val=line.split('\"')[0]
            text_val=line.split('\"')[1]
            for key, value in str_dict.items():
                if text_val == value[0] and not text_val.startswith("@string/"):
                    print(line)
                    line = line.replace(type_val+'\"'+text_val+'\"' , type_val+"\"@string/"+string_object+"/"+key+'\"')
                    print(line)
        newlines.append(line)
    code = open(file,'w+t', encoding='UTF8')
    code.writelines(newlines)
    code.close()

In [123]:
file = r'C:\Users\Tmax\Desktop\Hyperdata_web_TOP3\meta\layout\com\tmaxos\toptmax\layout\IntroductionLayout.tlf'
code = open(file,'r+',encoding='UTF8')
lines = code.readlines()
code.close()

In [124]:
lines

['<resource:LinearLayout\n',
 '    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"\n',
 '    xmlns:resource="http://www.tmaxsoft.com/top/SNAPSHOT/resource"\n',
 '    id="IntroductionLayout"\n',
 '    ideUniqueId="ID0a18aa56c0e43cb68c168d82f3303b66"\n',
 '    backgroundColor="#e4e9f0"\n',
 '    layoutHeight="match_parent"\n',
 '    webController="IntroductionLayoutLogic"\n',
 '    webControllerJs="src/common/IntroductionLayoutLogic.js"\n',
 '    orientation="vertical">\n',
 '        <resource:LinearLayout\n',
 '        id="LNB2Layout"\n',
 '        backgroundColor="#FFFFFF"\n',
 '        className="lnb01-bar"\n',
 '        layoutHeight="45"\n',
 '        layoutHorizontalAlignment="LEFT"\n',
 '        layoutWidth="match_parent"\n',
 '        orientation="horizontal">\n',
 '            <resource:LinearLayout\n',
 '            id="IntroductionTextViewLayout"\n',
 '            layoutHeight="match_parent"\n',
 '            layoutVerticalAlignment="BOTTOM"\n',
 '            layoutWidth=

In [136]:
path = r'C:\Users\Tmax\Desktop\Hyperdata_web_TOP3'  

In [137]:
#String object 읽어오기
string_object_path=path+'\\res'
string_object_name='str'
string_file_list= get_file_list(string_object_path, string_object_name+'.xml')
files = []
for file in string_file_list:
    files.append(file[0]+'//'+file[2])

str_dict = extract_match_string(files)

In [133]:
str_dict

defaultdict(list,
            {'str_name': ['Name', '이름'],
             'str_team': ['Team', '팀'],
             'str_user': ['User', '사용자'],
             'str_age': ['Age', '나이'],
             'String000004': ['  Bar Chart', '번역필요1'],
             'String000005': ['Measure Comparison', '번역필요2'],
             'String000006': ['Dimension Comparison', '번역필요3'],
             'String000007': ['  Line Chart', '번역필요4'],
             'String000008': ['  Pie Chart', '번역필요7'],
             'String000009': ['   Table', '번역필요10'],
             'String000010': ['Raw Data', '번역필요12'],
             'String000011': ['  html Card', '번역필요13'],
             'String000012': ['Data Source', '번역필요14'],
             'String000013': ['Virtual Model', '번역필요15'],
             'String000014': ['menuitem', '번역필요16'],
             'String000015': ['Shared Data', '번역필요17'],
             'String000016': ['ServiceJob', '번역필요18'],
             'String000017': ['DataJob', '번역필요19'],
             'String000018': ['Logou

In [134]:
#String id 및 value 읽어오기
def read_string_file(file_path):
    string_list = []
    code = open(file_path, 'rt', encoding='UTF8')
    while True:
        line = code.readline()
        if not line: break
        line = line.strip()
        if line.startswith("name="):
            str_id = line.split("\"")[1]
            str_val = line.split(str_id+'\">')[1].split('</resource:String>')[0]
            string_list.append([str_id,str_val])
        else: pass
    code.close()
        
    return string_list

In [139]:
for line in lines:
    if 'text=\"' in line or 'hint=\"' in line or 'message=\"' in line or 'title=\"' in line :
        type_val=line.split('\"')[0]
        text_val=line.split('\"')[1]
        print([text_val])
        for key, value in str_dict.items():
            if text_val == value[0] and not text_val.startswith("@string/"):
                print(line)
                line = line.replace(type_val+'\"'+text_val+'\"' , type_val+"\"@string/"+'str'"/"+key+'\"')
                print(line)

['@string/str/String000641']
['@string/str/String000745']
['@string/str/String000746']
['@string/str/String000747']
['새 프로젝트 생성 >']
                            text="새 프로젝트 생성 >"

                            text="@string/str/String000748"

['@string/str/String000382']
['@string/str/String000749']
['@string/str/String000313']
['@string/str/String000750']
['@string/str/String000642']
['@string/str/String000751']
['@string/str/String000643']
['@string/str/String000752']
